In [ ]:
# Global CSS for Voila dashboard
from IPython.display import display, HTML

display(HTML('''
<style>
/* Force white text on dark header buttons */
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.widget-button,
button.jupyter-button,
.lm-Widget button,
.jp-Button button {
    color: #fff !important;
}

.header-sort-button * {
    color: #fff !important;
}

/* No gap between elements */
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}
</style>
'''))

# TLS adapter for legacy SSL
import ssl
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = create_urllib3_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        ctx.set_ciphers('DEFAULT:@SECLEVEL=1')
        try:
            ctx.options |= ssl.OP_LEGACY_SERVER_CONNECT
        except AttributeError:
            pass
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

if not getattr(requests.Session, '_tls_patched', False):
    _orig_init = requests.Session.__init__
    def _patched_init(self, *args, **kwargs):
        _orig_init(self, *args, **kwargs)
        self.mount('https://', TLSAdapter())
        self.verify = False
    requests.Session.__init__ = _patched_init
    requests.Session._tls_patched = True

In [ ]:
%%capture
%pip install -q pandas xnat

import pandas as pd
import xnat
import os
import re
import tempfile
from pathlib import Path
from IPython.display import display

# Configuration
PROJECT_ID = os.environ.get('XNAT_PROJECT', 'YOUR_PROJECT_ID')
RESOURCE_NAME = os.environ.get('DATA_REQUEST_RESOURCE', 'data-request')

# Connect to XNAT
session = xnat.connect(
    server=os.environ.get('XNAT_HOST'),
    user=os.environ.get('XNAT_USER'),
    password=os.environ.get('XNAT_PASS'),
    verify=False
)
project = session.projects[PROJECT_ID]

In [ ]:
# Load DataRequest CSV files from project resources
def load_data_request_csvs(project, resource_name='data-request'):
    """Load all DataRequest CSV files from XNAT project resource."""
    study_pattern = re.compile(r'^DataRequestStudyStatus(\d+)_(\d+)\.csv$')
    series_pattern = re.compile(r'^DataRequestSeriesStatus(\d+)_(\d+)\.csv$')
    
    study_dfs, series_dfs = [], []
    
    if resource_name not in project.resources:
        print(f"Warning: Resource '{resource_name}' not found in project. Available: {list(project.resources.keys())}")
        return pd.DataFrame(), pd.DataFrame()
    
    resource = project.resources[resource_name]
    cache_dir = Path(tempfile.gettempdir()) / 'data_request_review_cache'
    cache_dir.mkdir(exist_ok=True)
    
    for filename in resource.files.keys():
        study_match = study_pattern.match(filename)
        series_match = series_pattern.match(filename)
        
        if study_match or series_match:
            match = study_match or series_match
            cache_path = cache_dir / filename
            
            # Always re-download to avoid stale/incomplete cache issues
            # Download to temp file first, then move to ensure atomicity
            temp_path = cache_path.with_suffix('.tmp')
            try:
                resource.files[filename].download(str(temp_path))
                # Verify file has content before using it
                if temp_path.stat().st_size > 0:
                    temp_path.replace(cache_path)
                else:
                    raise ValueError(f"Downloaded file {filename} is empty")
            except Exception as e:
                # If download fails but cache exists, use cached version
                if cache_path.exists() and cache_path.stat().st_size > 0:
                    pass  # Use existing cache
                else:
                    raise e
            finally:
                # Clean up temp file if it exists
                if temp_path.exists():
                    temp_path.unlink()
            
            df = pd.read_csv(cache_path)
            df['_data_request_id'] = int(match.group(1))
            df['_execution_time'] = match.group(2)
            
            if study_match:
                study_dfs.append(df)
            else:
                series_dfs.append(df)
    
    def dedupe_by_request(dfs):
        """Keep only latest execution per data request."""
        if not dfs:
            return pd.DataFrame()
        combined = pd.concat(dfs, ignore_index=True)
        latest = combined.groupby('_data_request_id')['_execution_time'].transform('max')
        return combined[combined['_execution_time'] == latest]
    
    def dedupe_studies(df):
        """Deduplicate studies across all data requests.
        
        1. Dedupe by studyInstanceUid - if same study appears in multiple requests,
           keep only the latest entry (by execution time), regardless of status.
        2. Dedupe AVAILABLE by experimentLabel - if different study UIDs archived to
           the same XNAT experiment, count as one.
        """
        if df.empty:
            return df
        
        df = df.copy()
        df['status'] = df['status'].str.upper()
        
        # Sort by execution time descending (latest first)
        df = df.sort_values('_execution_time', ascending=False)
        
        # Dedupe by studyInstanceUid - keeps latest entry regardless of status
        if 'studyInstanceUid' in df.columns:
            df = df.drop_duplicates(subset='studyInstanceUid', keep='first')
        
        # Dedupe AVAILABLE by experimentLabel (different study UIDs → same experiment)
        available = df[df['status'] == 'AVAILABLE'].copy()
        other = df[df['status'] != 'AVAILABLE'].copy()
        
        if not available.empty and 'experimentLabel' in available.columns:
            has_label = available['experimentLabel'].notna()
            available_with_label = available[has_label].drop_duplicates(subset='experimentLabel', keep='first')
            available_without_label = available[~has_label]
            available = pd.concat([available_with_label, available_without_label], ignore_index=True)
        
        return pd.concat([available, other], ignore_index=True)
    
    def dedupe_series(df):
        """Deduplicate series by seriesInstanceUid, keeping the latest entry."""
        if df.empty or 'seriesInstanceUid' not in df.columns:
            return df
        df = df.sort_values('_execution_time', ascending=False)
        return df.drop_duplicates(subset='seriesInstanceUid', keep='first')
    
    # First dedupe by request (keep latest execution per request)
    studies = dedupe_by_request(study_dfs)
    series = dedupe_by_request(series_dfs)
    
    # Then dedupe across requests
    studies = dedupe_studies(studies)
    series = dedupe_series(series)
    
    return studies, series

try:
    study_df, series_df = load_data_request_csvs(project, RESOURCE_NAME)
except Exception as e:
    from ipywidgets import HTML
    display(HTML(f'''
        <div style="background: #fdeaea; border: 1px solid #e74c3c; border-radius: 6px; padding: 15px; margin: 10px 0; color: #721c24;">
            <strong>Error loading data requests:</strong><br>
            <code>{type(e).__name__}: {e}</code>
        </div>
    '''))
    study_df, series_df = pd.DataFrame(), pd.DataFrame()

In [ ]:
# Data Request Overview
import ipywidgets as widgets
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Dropdown
import html as html_module
import matplotlib.pyplot as plt
import warnings

# Suppress xnat warnings
warnings.filterwarnings('ignore')

# Global state for series data (accessible by other cells)
scans_by_experiment = {}
series_data_loaded = False
missing_series_output = None  # Will be set below, displayed in validation cell

def clean_xsi_type(xsi_type, suffix='SessionData'):
    """Clean xsiType: strip 'xnat:' prefix and suffix, uppercase result.

    Examples:
        clean_xsi_type('xnat:mrSessionData') -> 'MR'
        clean_xsi_type('xnat:ctScanData', 'ScanData') -> 'CT'
    """
    if not xsi_type or xsi_type == 'Unknown':
        return 'Unknown'

    result = str(xsi_type)

    # Strip xnat: prefix
    if result.startswith('xnat:'):
        result = result[5:]

    # Strip the suffix (SessionData or ScanData)
    if result.endswith(suffix):
        result = result[:-len(suffix)]
        return result.upper()
    else:
        # Doesn't match pattern, return as-is
        return xsi_type

def stat_card(label, value, color, tooltip=''):
    """Create a uniformly-sized status card widget with optional tooltip."""
    title_attr = f'title="{tooltip}"' if tooltip else ''
    return HTML(f'''
        <div {title_attr} style="
            background: linear-gradient(135deg, {color}22, {color}11);
            border-left: 4px solid {color};
            border-radius: 8px;
            padding: 15px 20px;
            margin: 5px;
            min-width: 140px;
            width: 140px;
            height: 70px;
            text-align: center;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            display: flex;
            flex-direction: column;
            justify-content: center;
            cursor: help;
        ">
            <div style="font-size: 28px; font-weight: bold; color: {color};">{value:,}</div>
            <div style="font-size: 11px; color: #666; text-transform: uppercase; letter-spacing: 1px;">{label}</div>
        </div>
    ''')

# Get experiment count without triggering lazy loading
exp_response = session.get(f'/data/projects/{PROJECT_ID}/experiments', query={'format': 'json', 'columns': 'ID,label,xsiType'})
xnat_experiments = exp_response.json().get('ResultSet', {}).get('Result', [])
project_experiment_count = len(xnat_experiments)
xnat_exp_labels = {e['label'] for e in xnat_experiments}
exp_map = {e['label']: e['ID'] for e in xnat_experiments}
exp_xsi_map = {e['label']: e.get('xsiType', 'Unknown') or 'Unknown' for e in xnat_experiments}

if not study_df.empty:
    study_df['status'] = study_df['status'].str.upper()
    
    total = len(study_df)
    available = (study_df['status'] == 'AVAILABLE').sum()
    errors = (study_df['status'] == 'ERROR').sum()
    rejected = (study_df['status'] == 'REJECTED').sum()
    unique_accessions = study_df['accessionNumber'].nunique() if 'accessionNumber' in study_df.columns else 0
    extra_studies = total - unique_accessions
    
    # Calculate missing from project (AVAILABLE but not in XNAT)
    available_df = study_df[study_df['status'] == 'AVAILABLE'].copy()
    if 'experimentLabel' in available_df.columns:
        def is_missing_from_project(row):
            exp_label = row.get('experimentLabel')
            if pd.isna(exp_label) or exp_label == '' or exp_label is None:
                return True
            return str(exp_label).strip() not in xnat_exp_labels
        missing_from_project_count = available_df.apply(is_missing_from_project, axis=1).sum()
    else:
        missing_from_project_count = 0
    
    # Find accession numbers with multiple studies (for use in next cell)
    if 'accessionNumber' in study_df.columns:
        accession_counts = study_df.groupby('accessionNumber').size()
        multi_accessions = accession_counts[accession_counts > 1].index.tolist()
        multi_study_accessions_count = len(multi_accessions)
        
        if multi_accessions:
            multi_study_df = study_df[study_df['accessionNumber'].isin(multi_accessions)].copy()
            multi_study_df = multi_study_df.sort_values(['accessionNumber', 'studyInstanceUid'])
        else:
            multi_study_df = pd.DataFrame()
    else:
        multi_study_accessions_count = 0
        multi_study_df = pd.DataFrame()
    
    # Main header
    header = HTML('<h1 style="margin: 0 0 20px 5px; color: #333; font-size: 32px; font-weight: 600;">Data Request Review</h1>')
    
    # All cards in one row (Studies In Project first)
    cards = [
        stat_card('Studies In Project', project_experiment_count, '#1abc9c', 'Actual studies in the project'),
        stat_card('Available Studies', available, '#27ae60', 'Studies available in the data request'),
        stat_card('Requested Studies', total, '#3498db', 'Unique Study Instance UIDs'),
        stat_card('Accession Numbers', unique_accessions, '#9b59b6', 'Unique Accession Numbers'),
        stat_card('Error Studies', errors, '#e74c3c', 'Studies not available due to error'),
        stat_card('Missing Studies', missing_from_project_count, '#9c27b0', 'Available studies missing from project'),
        stat_card('Excluded Studies', rejected, '#f39c12', 'Studies rejected by Site Anonymization Script'),
    ]
    
    cards_row = HBox(cards, layout=Layout(flex_flow='row wrap'))
    
    display(VBox([header, cards_row], layout=Layout(padding='10px')))
    
    # Show green banner if no extra studies
    if extra_studies == 0 and unique_accessions > 0:
        note = HTML(f'''
            <div style="
                background: #d4edda;
                border: 1px solid #28a745;
                border-radius: 6px;
                padding: 10px 15px;
                margin: 10px 5px;
                font-size: 13px;
                color: #000;
                width: 1400px;
                box-sizing: border-box;
            ">
                Each accession number maps to exactly one study.
            </div>
        ''')
        display(note)
    
    # ==================== SESSION TYPES PIE CHART (loads immediately) ====================
    available_in_xnat = available_df[available_df['experimentLabel'].isin(exp_map.keys())]
    session_types_from_xnat = []
    
    for _, row in available_in_xnat.iterrows():
        exp_label = row.get('experimentLabel', '')
        if pd.isna(exp_label) or exp_label not in exp_map:
            continue
        xsi_type = exp_xsi_map.get(exp_label, 'Unknown')
        cleaned_type = clean_xsi_type(xsi_type, 'SessionData')
        session_types_from_xnat.append(cleaned_type)
    
    has_session_types = len(session_types_from_xnat) > 0
    
    # Create session types chart output (will be shown alone initially, then side-by-side)
    session_chart_output = Output()
    with session_chart_output:
        if has_session_types:
            fig, ax = plt.subplots(1, 1, figsize=(6, 4.5))
            
            session_type_counts = pd.Series(session_types_from_xnat).value_counts()
            total_sessions = len(session_types_from_xnat)
            colors = plt.cm.tab20(range(len(session_type_counts)))
            
            wedges, _ = ax.pie(
                session_type_counts.values,
                startangle=90,
                colors=colors,
                wedgeprops=dict(width=0.5, edgecolor='white')
            )
            
            ax.text(0, 0, f'{total_sessions:,}\nSessions', ha='center', va='center', 
                    fontsize=14, fontweight='bold', color='#333')
            
            legend_labels = [f'{stype} - {count:,} ({count/total_sessions*100:.1f}%)' 
                             for stype, count in zip(session_type_counts.index, session_type_counts.values)]
            ax.legend(wedges, legend_labels, title='Type', loc='center left', 
                      bbox_to_anchor=(1.05, 0.5), fontsize=9, title_fontsize=10)
            ax.set_title('Session Types', fontsize=14, fontweight='bold', pad=15)
            
            plt.tight_layout()
            plt.show()
    
    # Scan types chart output (will be populated after loading)
    scan_chart_output = Output()
    
    # Container for both charts side by side
    charts_container = HBox([session_chart_output], layout=Layout(justify_content='flex-start', align_items='flex-start'))
    display(charts_container)
    
    # ==================== SERIES ANALYSIS SECTION ====================
    XNAT_HOST = os.environ.get('XNAT_HOST', '')
    
    load_series_btn = Button(
        description='Load Series Data',
        button_style='warning',
        tooltip='Fetch series data from XNAT to view scan type breakdown and missing series analysis',
        icon='download',
        layout=Layout(margin='0 15px 0 0')
    )
    
    series_output = Output()
    progress_html = HTML('<span style="color: #856404;">Click button to load series data for scan type breakdown and missing series analysis</span>')
    
    # Yellow warning banner with button inside
    series_banner_content = HBox([load_series_btn, progress_html], layout=Layout(align_items='center'))
    series_banner = VBox([series_banner_content], layout=Layout(
        background='#fff8e6',
        border='1px solid #f39c12',
        border_radius='10px',
        padding='15px 20px',
        margin='10px 5px',
        width='1400px'
    ))
    
    # Missing series output (shown after loading, displayed in validation cell at bottom)
    missing_series_output = Output()
    
    # SOP classes excluded by site anonymization script
    EXCLUDED_SOP_CLASSES = {
        '1.2.840.10008.5.1.4.1.1.7': 'Secondary Capture Image',
        '1.2.840.10008.5.1.4.1.1.7.1': 'Multi-frame Single Bit SC Image',
        '1.2.840.10008.5.1.4.1.1.7.2': 'Multi-frame Grayscale Byte SC Image',
        '1.2.840.10008.5.1.4.1.1.7.3': 'Multi-frame Grayscale Word SC Image',
        '1.2.840.10008.5.1.4.1.1.7.4': 'Multi-frame True Color SC Image',
        '1.2.840.10008.5.1.4.1.1.104.1': 'Encapsulated PDF',
        '1.2.840.10008.5.1.4.1.1.104.2': 'Encapsulated CDA',
    }
    
    def sop_with_tooltip(sop_uid):
        sop_uid = str(sop_uid).strip() if sop_uid else ''
        if not sop_uid or sop_uid == 'nan':
            return ''
        if sop_uid in EXCLUDED_SOP_CLASSES:
            name = EXCLUDED_SOP_CLASSES[sop_uid]
            return f'<span title="{html_module.escape(sop_uid)}" style="cursor: help;">{html_module.escape(name)}</span>'
        return sop_uid
    
    def on_load_series(b):
        global scans_by_experiment, series_data_loaded
        
        total_experiments = len(exp_map)
        
        progress_html.value = '<span style="color: #856404;">Connecting to XNAT...</span>'
        
        try:
            import logging
            logging.getLogger('xnat').setLevel(logging.ERROR)
            
            new_session = xnat.connect(
                server=os.environ.get('XNAT_HOST'),
                user=os.environ.get('XNAT_USER'),
                password=os.environ.get('XNAT_PASS'),
                verify=False
            )
        except Exception as e:
            progress_html.value = f'<span style="color: #c00;">Failed to connect: {e}</span>'
            return
        
        # Fetch scan data using REST API with xsiType column
        errors_list = []
        exp_items = list(exp_map.items())
        
        for i, (exp_label, exp_id) in enumerate(exp_items):
            progress_html.value = f'<span style="color: #856404;">Loading series data from experiment {i + 1:,} / {total_experiments:,}...</span>'
            
            try:
                scans_response = new_session.get(f'/data/experiments/{exp_id}/scans', query={'format': 'json', 'columns': 'ID,UID,series_description,xsiType'})
                scans_data = scans_response.json().get('ResultSet', {}).get('Result', [])
                scans_by_experiment[exp_label] = [
                    {
                        'id': scan.get('ID', ''),
                        'uid': scan.get('UID', ''),
                        'series_description': scan.get('series_description', ''),
                        'xsiType': scan.get('xsiType', 'Unknown') or 'Unknown'
                    }
                    for scan in scans_data
                ]
            except Exception as e:
                scans_by_experiment[exp_label] = []
                if len(errors_list) < 3:
                    errors_list.append(f"{exp_label}: {type(e).__name__}: {e}")
        
        series_data_loaded = True
        
        # Build scan type data
        all_project_scans = []
        for exp_label, scans in scans_by_experiment.items():
            for scan in scans:
                cleaned_type = clean_xsi_type(scan['xsiType'], 'ScanData')
                all_project_scans.append({'type': cleaned_type})
        
        exps_with_scans = sum(1 for scans in scans_by_experiment.values() if scans)
        total_scans = len(all_project_scans)
        
        # Update button to show loaded state
        load_series_btn.disabled = True
        load_series_btn.description = 'Series Loaded'
        load_series_btn.button_style = ''
        load_series_btn.icon = 'check'
        load_series_btn.style.button_color = '#6c757d'
        
        progress_html.value = f'<span style="color: #155724;">✓ <strong>Loaded {total_scans:,} series from {exps_with_scans:,} experiments.</strong> Scroll down to view Missing Series Analysis.</span>'
        
        # Update banner color to green
        series_banner.layout.background = '#d4edda'
        series_banner.layout.border = '1px solid #28a745'
        
        # Create scan types chart
        with scan_chart_output:
            scan_chart_output.clear_output(wait=True)
            if all_project_scans:
                fig, ax = plt.subplots(1, 1, figsize=(6, 4.5))
                
                scan_type_counts = pd.DataFrame(all_project_scans)['type'].value_counts()
                total_series = len(all_project_scans)
                colors = plt.cm.tab20(range(len(scan_type_counts)))
                
                wedges, _ = ax.pie(
                    scan_type_counts.values,
                    startangle=90,
                    colors=colors,
                    wedgeprops=dict(width=0.5, edgecolor='white')
                )
                
                ax.text(0, 0, f'{total_series:,}\nScans', ha='center', va='center', 
                        fontsize=14, fontweight='bold', color='#333')
                
                legend_labels = [f'{stype} - {count:,} ({count/total_series*100:.1f}%)' 
                                 for stype, count in zip(scan_type_counts.index, scan_type_counts.values)]
                ax.legend(wedges, legend_labels, title='Type', loc='center left', 
                          bbox_to_anchor=(1.05, 0.5), fontsize=9, title_fontsize=10)
                ax.set_title('Scan Types', fontsize=14, fontweight='bold', pad=15)
                
                plt.tight_layout()
                plt.show()
        
        # Add scan chart next to session chart
        charts_container.children = [session_chart_output, scan_chart_output]
        
        # ==================== BUILD MISSING SERIES WIDGET ====================
        XNAT_HOST = os.environ.get('XNAT_HOST', '')
        studies_with_missing = []
        
        available = study_df[study_df['status'] == 'AVAILABLE']
        series_by_exp = series_df.groupby('experimentLabel') if not series_df.empty and 'experimentLabel' in series_df.columns else None
        
        for _, row in available.iterrows():
            exp_label = row.get('experimentLabel', '')
            if pd.isna(exp_label) or exp_label not in exp_map:
                continue
            
            xnat_scans = scans_by_experiment.get(exp_label, [])
            actual_count = len(xnat_scans)
            xnat_series_uids = {scan.get('uid', '').strip() for scan in xnat_scans if scan.get('uid')}
            
            if series_by_exp is not None and exp_label in series_by_exp.groups:
                expected = series_by_exp.get_group(exp_label)
                expected_count = len(expected)

                def is_missing(r):
                    uid = str(r.get('seriesInstanceUid', '')).strip()
                    if not uid or uid == 'nan':
                        return True
                    return uid not in xnat_series_uids

                all_missing_series = expected[expected.apply(is_missing, axis=1)].copy()

                if len(all_missing_series) > 0:
                    excluded_series = all_missing_series[all_missing_series['sopClassUid'].isin(EXCLUDED_SOP_CLASSES.keys())].copy()
                    actually_missing = all_missing_series[~all_missing_series['sopClassUid'].isin(EXCLUDED_SOP_CLASSES.keys())].copy()
                    
                    excluded_count = len(excluded_series)
                    unexplained_count = len(actually_missing)
                    
                    if unexplained_count > 0:
                        study_uid = row.get('studyInstanceUid', '')
                        if pd.isna(study_uid):
                            study_uid = ''
                        studies_with_missing.append({
                            'experimentLabel': exp_label,
                            'accessionNumber': row.get('accessionNumber', ''),
                            'studyInstanceUid': study_uid,
                            'expected': expected_count,
                            'actual': actual_count,
                            'missing': excluded_count + unexplained_count,
                            'excluded_sop': excluded_count,
                            'unexplained': unexplained_count,
                            'excluded_series': excluded_series,
                            'actually_missing': actually_missing
                        })
        
        with missing_series_output:
            missing_series_output.clear_output(wait=True)
            
            if studies_with_missing:
                total_missing_studies = len(studies_with_missing)
                total_excluded = sum(s['excluded_sop'] for s in studies_with_missing)
                total_unexplained = sum(s['unexplained'] for s in studies_with_missing)
                
                ms_header = HTML(f'''
                    <div style="
                        background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
                        color: white;
                        padding: 20px;
                        border-radius: 10px 10px 0 0;
                    ">
                        <h3 style="margin: 0; font-size: 18px;">Missing Series</h3>
                        <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{total_missing_studies} studies with missing series</p>
                    </div>
                ''')
                
                ms_summary = HTML(f'''
                    <div style="display: flex; gap: 15px; padding: 15px 20px; background: #f8f9fa;">
                        <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                            <div style="font-size: 24px; font-weight: bold; color: #e74c3c;">{total_missing_studies}</div>
                            <div style="font-size: 11px; color: #666; text-transform: uppercase;">Studies Affected</div>
                        </div>
                        <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                            <div style="font-size: 24px; font-weight: bold; color: #f39c12;">{total_excluded}</div>
                            <div style="font-size: 11px; color: #666; text-transform: uppercase;">Excluded</div>
                        </div>
                        <div style="text-align: center; padding: 10px 20px; background: white; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.1);">
                            <div style="font-size: 24px; font-weight: bold; color: #c0392b;">{total_unexplained}</div>
                            <div style="font-size: 11px; color: #666; text-transform: uppercase;">Missing</div>
                        </div>
                    </div>
                ''')
                
                # Build options list (no placeholder, start with first session)
                options = []
                for s in studies_with_missing:
                    label = f"{s['experimentLabel']} — {s['missing']} missing ({s['excluded_sop']} excluded, {s['unexplained']} missing)"
                    options.append((label, s))
                
                # Create dropdown with first session selected
                ms_dropdown = Dropdown(options=options, value=studies_with_missing[0], layout=Layout(width='100%'))
                ms_detail_output = Output()
                
                def on_ms_select(change):
                    ms_detail_output.clear_output()
                    study = change['new']
                    if study is None:
                        return
                    
                    with ms_detail_output:
                        xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{study['experimentLabel']}"
                        
                        display(HTML(f'''
                            <div style="background: white; padding: 20px;">
                                <div style="display: flex; justify-content: space-between; align-items: center;">
                                    <div>
                                        <div style="font-size: 16px; font-weight: bold;">
                                            <a href="{xnat_url}" target="_blank" style="color: #e74c3c; text-decoration: none;">{study['experimentLabel']}</a>
                                        </div>
                                        <div style="font-size: 12px; color: #888; margin-top: 4px;">Accession: {study['accessionNumber']}</div>
                                        <div style="font-size: 11px; color: #aaa; margin-top: 2px;">Study Instance UID: {study['studyInstanceUid']}</div>
                                    </div>
                                    <div style="display: flex; gap: 10px;">
                                        <div style="text-align: center; padding: 8px 15px; background: #e8f4fd; border-radius: 6px;">
                                            <div style="font-size: 18px; font-weight: bold; color: #3498db;">{study['expected']}</div>
                                            <div style="font-size: 10px; color: #666;">Expected</div>
                                        </div>
                                        <div style="text-align: center; padding: 8px 15px; background: #e8f8e8; border-radius: 6px;">
                                            <div style="font-size: 18px; font-weight: bold; color: #27ae60;">{study['actual']}</div>
                                            <div style="font-size: 10px; color: #666;">In XNAT</div>
                                        </div>
                                        <div style="text-align: center; padding: 8px 15px; background: #fff8e6; border-radius: 6px;">
                                            <div style="font-size: 18px; font-weight: bold; color: #f39c12;">{study['excluded_sop']}</div>
                                            <div style="font-size: 10px; color: #856404;">Excluded</div>
                                        </div>
                                        <div style="text-align: center; padding: 8px 15px; background: #fdeaea; border-radius: 6px;">
                                            <div style="font-size: 18px; font-weight: bold; color: #e74c3c;">{study['unexplained']}</div>
                                            <div style="font-size: 10px; color: #721c24;">Missing</div>
                                        </div>
                                    </div>
                                </div>
                            </div>
                        '''))
                        
                        all_missing = []
                        is_excluded_flags = []
                        
                        for _, r in study['excluded_series'].iterrows():
                            all_missing.append({
                                'Series UID': r.get('seriesInstanceUid', ''),
                                'Status': 'Excluded',
                                'Series Description': r.get('seriesDescription', ''),
                                'Modality': r.get('modality', ''),
                                'SOP Class': sop_with_tooltip(r.get('sopClassUid', ''))
                            })
                            is_excluded_flags.append(True)
                        
                        for _, r in study['actually_missing'].iterrows():
                            all_missing.append({
                                'Series UID': r.get('seriesInstanceUid', ''),
                                'Status': 'Missing',
                                'Series Description': r.get('seriesDescription', ''),
                                'Modality': r.get('modality', ''),
                                'SOP Class': sop_with_tooltip(r.get('sopClassUid', ''))
                            })
                            is_excluded_flags.append(False)
                        
                        if all_missing:
                            display_df = pd.DataFrame(all_missing)
                            def style_row(row_idx):
                                if is_excluded_flags[row_idx]:
                                    return ['background-color: #fff8e6; color: #856404;'] * len(display_df.columns)
                                else:
                                    return ['background-color: #fdeaea; color: #721c24;'] * len(display_df.columns)

                            styled = display_df.style.apply(lambda x: style_row(x.name), axis=1)\
                                .set_properties(**{'text-align': 'left', 'font-size': '12px', 'padding': '10px'})\
                                .set_table_styles([
                                    {'selector': '', 'props': [('width', '100%'), ('border-collapse', 'collapse')]},
                                    {'selector': 'th', 'props': [('background-color', '#f8f9fa'), ('font-size', '11px'), ('text-transform', 'uppercase'), ('color', '#666'), ('padding', '10px'), ('border-bottom', '2px solid #ddd')]},
                                    {'selector': 'td', 'props': [('border-bottom', '1px solid #eee')]},
                                ]).hide(axis='index')

                            table_html = styled.to_html()
                            table_html = table_html.replace('&lt;span title=', '<span title=').replace('style=&quot;cursor: help;&quot;&gt;', 'style="cursor: help;">').replace('&lt;/span&gt;', '</span>').replace('&quot;', '"')
                            display(HTML(f'<div style="background: white; padding: 15px 20px;">{table_html}</div>'))
                
                ms_dropdown.observe(on_ms_select, names='value')
                
                # Label for dropdown
                ms_dropdown_label = HTML('<div style="font-size: 12px; font-weight: bold; color: #666; margin-bottom: 5px;">Select Study</div>')
                
                ms_widget = VBox([
                    ms_header,
                    ms_summary,
                    VBox([ms_dropdown_label, ms_dropdown], layout=Layout(padding='15px 20px', width='100%', overflow='hidden')),
                    ms_detail_output
                ], layout=Layout(width='1400px', border_radius='10px', overflow='hidden', background='#f8f9fa', margin='10px 5px'))
                
                display(ms_widget)
                
                # Trigger initial display of first session
                on_ms_select({'new': studies_with_missing[0]})
            else:
                display(HTML('''
                    <div style="background: linear-gradient(135deg, #56ab2f 0%, #a8e063 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; width: 1400px; margin: 10px 5px;">
                        <h3 style="margin: 0;">All Series Complete</h3>
                        <p style="margin: 10px 0 0 0; opacity: 0.9;">No missing series detected.</p>
                    </div>
                '''))
    
    load_series_btn.on_click(on_load_series)
    
    display(series_banner)
    # Note: missing_series_output is displayed in the validation cell at the bottom

else:
    print("No study data available.")

In [ ]:
# Additional Studies (accessions with multiple study UIDs)
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Text
import ipywidgets as widgets
import html as html_module

# Get XNAT host for building links
XNAT_HOST = os.environ.get('XNAT_HOST', '')

def truncate_with_tooltip(text, max_len=50):
    """Truncate text and add tooltip if needed."""
    text = str(text)
    if len(text) <= max_len:
        return html_module.escape(text)
    truncated = text[:max_len].rsplit(' ', 1)[0] + '...'
    escaped_full = html_module.escape(text)
    escaped_truncated = html_module.escape(truncated)
    return f'<span title="{escaped_full}" style="cursor: help;">{escaped_truncated}</span>'

# CSS to fix button styling - force white text on dark background for both light/dark themes
add_button_style_css = HTML('''
<style>
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.header-sort-button button .bp3-button-text,
.header-sort-button button span,
.header-sort-button button p {
    border-radius: 0 !important;
    font-weight: bold !important;
    font-size: 11px !important;
    text-transform: uppercase !important;
    border: none !important;
    border-bottom: 2px solid #555 !important;
    background-color: #333 !important;
    color: #fff !important;
}
.header-sort-button button:hover {
    background-color: #444 !important;
}
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}
</style>
''')
display(add_button_style_css)

# ==================== ADDITIONAL STUDIES TABLE ====================
if not study_df.empty and extra_studies > 0 and not multi_study_df.empty:
    additional_details = []
    for _, row in multi_study_df.iterrows():
        exp_label = row.get('experimentLabel', '')
        if pd.isna(exp_label):
            exp_label = ''
        
        study_uid = str(row.get('studyInstanceUid', '')).strip()
        if not study_uid or study_uid == 'nan' or study_uid == 'None':
            study_uid = 'Unknown'
        
        status = row.get('status', '')
        if status == 'ERROR':
            reason = str(row.get('statusMessage', '')).strip() or 'Unknown'
        elif status == 'REJECTED':
            reason = str(row.get('statusMessage', '')).strip() or 'Rejected by site'
        else:
            reason = ''
        
        additional_details.append({
            'Experiment Label': str(exp_label),
            'Accession Number': str(row.get('accessionNumber', '')),
            'Study UID': study_uid,
            'Status': status,
            'Reason': reason,
        })
    
    additional_table_df = pd.DataFrame(additional_details)
    
    PAGE_SIZE = 15
    add_state = {'page': 0, 'sort_col': 'Accession Number', 'sort_asc': True, 'filter': ''}
    
    # Column definitions - wider accession/UID, smaller reason
    add_columns = [
        ('Experiment Label', '12%'),
        ('Accession Number', '15%'),
        ('Study UID', '30%'),
        ('Status', '8%'),
        ('Reason', '35%'),
    ]
    
    add_filter_input = Text(
        placeholder='Search',
        layout=Layout(width='300px')
    )
    
    add_prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
    add_next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
    add_page_label = HTML()
    add_table_output = Output()
    
    # Create header buttons list first
    add_header_buttons = []
    
    def update_add_header_buttons():
        """Update header button labels with sort arrows."""
        for i, (col_name, width) in enumerate(add_columns):
            arrow = ''
            if add_state['sort_col'] == col_name:
                arrow = ' ↑' if add_state['sort_asc'] else ' ↓'
            add_header_buttons[i].description = f'{col_name}{arrow}'
    
    def make_add_sort_handler(col_name):
        def handler(b):
            if add_state['sort_col'] == col_name:
                add_state['sort_asc'] = not add_state['sort_asc']
            else:
                add_state['sort_col'] = col_name
                add_state['sort_asc'] = True
            add_state['page'] = 0
            update_add_header_buttons()
            render_additional_table()
        return handler
    
    def render_additional_table():
        add_table_output.clear_output(wait=True)
        
        # Apply filter across all columns
        filtered_df = additional_table_df.copy()
        if add_state['filter']:
            filter_pattern = add_state['filter'].replace('*', '.*')
            mask = (
                filtered_df['Experiment Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Accession Number'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Study UID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Status'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                filtered_df['Reason'].str.contains(filter_pattern, case=False, regex=True, na=False)
            )
            filtered_df = filtered_df[mask]
        
        sorted_df = filtered_df.sort_values(
            by=add_state['sort_col'],
            ascending=add_state['sort_asc'],
            key=lambda x: x.str.lower() if x.dtype == 'object' else x
        ).reset_index(drop=True)
        
        total_pages = max(1, (len(sorted_df) + PAGE_SIZE - 1) // PAGE_SIZE)
        add_state['page'] = max(0, min(add_state['page'], total_pages - 1))
        start = add_state['page'] * PAGE_SIZE
        end = start + PAGE_SIZE
        page_df = sorted_df.iloc[start:end].copy()
        
        filter_text = f" (filtered: {len(filtered_df)})" if add_state['filter'] else ""
        add_page_label.value = f'<span style="font-size: 13px; color: #666;">Page {add_state["page"] + 1} of {total_pages} ({len(sorted_df)} studies{filter_text})</span>'
        
        add_prev_btn.disabled = add_state['page'] == 0
        add_next_btn.disabled = add_state['page'] >= total_pages - 1
        
        # Build data rows
        rows = []
        for _, r in page_df.iterrows():
            exp_label = str(r['Experiment Label'])
            acc = html_module.escape(str(r['Accession Number']))
            uid = html_module.escape(str(r['Study UID']))
            stat = html_module.escape(str(r['Status']))
            reason = truncate_with_tooltip(str(r['Reason']), 50)
            
            # Color based on status
            status_upper = str(r['Status']).upper()
            if status_upper == 'ERROR':
                bg_color = '#fdeaea'
                text_color = '#721c24'
            elif status_upper == 'REJECTED':
                bg_color = '#fff8e6'
                text_color = '#856404'
            else:
                bg_color = '#f0f7ff'
                text_color = '#2c5282'
            
            # Create experiment label with link for AVAILABLE studies
            if status_upper == 'AVAILABLE' and exp_label and exp_label.strip():
                xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{exp_label}"
                exp_display = f'<a href="{xnat_url}" target="_blank" style="color: {text_color}; text-decoration: underline;">{html_module.escape(exp_label)}</a>'
            else:
                exp_display = html_module.escape(exp_label)
            
            rows.append(f'''<tr style="background-color: {bg_color}; color: {text_color};">
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 12%;">{exp_display}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 15%;">{acc}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 30%; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">{uid}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 8%;">{stat}</td>
                <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 35%;">{reason}</td>
            </tr>''')
        
        table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; table-layout: fixed; background: white; margin: 0; padding: 0;">
            <tbody>{"".join(rows)}</tbody>
        </table>'''
        
        with add_table_output:
            display(HTML(table_html))
    
    # Create header buttons for sorting - use dark background
    for col_name, width in add_columns:
        arrow = ''
        if add_state['sort_col'] == col_name:
            arrow = ' ↑' if add_state['sort_asc'] else ' ↓'
        
        btn = Button(description=f'{col_name}{arrow}')
        btn.layout = Layout(width=width, height='36px', margin='0', padding='0')
        btn.style.button_color = '#333'
        btn.add_class('header-sort-button')
        btn.on_click(make_add_sort_handler(col_name))
        add_header_buttons.append(btn)
    
    def on_add_filter_change(change):
        add_state['filter'] = change['new']
        add_state['page'] = 0
        render_additional_table()
    
    def on_add_prev(b):
        add_state['page'] -= 1
        render_additional_table()
    
    def on_add_next(b):
        add_state['page'] += 1
        render_additional_table()
    
    add_filter_input.observe(on_add_filter_change, names='value')
    add_prev_btn.on_click(on_add_prev)
    add_next_btn.on_click(on_add_next)
    
    render_additional_table()
    
    add_title_widget = HTML(f'''
        <div style="
            background: linear-gradient(135deg, #3498db 0%, #2980b9 100%);
            color: white;
            padding: 20px;
            border-radius: 10px 10px 0 0;
        ">
            <h3 style="margin: 0; font-size: 18px;">Additional Studies</h3>
            <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{extra_studies} additional studies found. {multi_study_accessions_count} accession numbers have multiple studies in the PACS</p>
        </div>
    ''')
    
    # Left side: search box
    add_left_controls = HBox([add_filter_input], layout=Layout(align_items='center'))
    
    # Right side: pagination
    add_right_controls = HBox([add_prev_btn, add_page_label, add_next_btn], layout=Layout(align_items='center', gap='10px'))
    
    # Controls row with space-between to push pagination to right
    add_controls = HBox(
        [add_left_controls, add_right_controls],
        layout=Layout(padding='15px 20px', justify_content='space-between', align_items='center', background='#f5f5f5', width='100%')
    )
    
    # Header row using HBox with buttons - dark background
    add_header_row = HBox(add_header_buttons)
    add_header_row.layout = Layout(width='100%', margin='0', padding='0', background='#333', overflow='hidden')
    add_header_row.add_class('no-gap')
    
    # Table output wrapper with explicit white background
    add_table_wrapper = VBox([add_table_output])
    add_table_wrapper.layout = Layout(background='white', padding='0', margin='0', width='100%', overflow='hidden')
    
    # Table container - header and table together with no gaps
    add_table_container = VBox([add_header_row, add_table_wrapper])
    add_table_container.layout = Layout(background='#333', padding='0', margin='0', width='100%', overflow='hidden')
    add_table_container.add_class('no-gap')
    
    add_full_widget = VBox([
        add_title_widget,
        add_controls,
        add_table_container,
    ])
    add_full_widget.layout = Layout(width='1400px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px')
    add_full_widget.add_class('no-gap')
    
    display(add_full_widget)

In [ ]:
# Study Errors and Missing
import ipywidgets as widgets
from ipywidgets import HTML, HBox, VBox, Layout, Button, Output, Text
import html as html_module

def truncate_with_tooltip(text, max_len=50):
    """Truncate text and add tooltip if needed."""
    text = str(text)
    if len(text) <= max_len:
        return text
    truncated = text[:max_len].rsplit(' ', 1)[0] + '...'
    escaped_full = html_module.escape(text)
    escaped_truncated = html_module.escape(truncated)
    return f'<span title="{escaped_full}" style="cursor: help;">{escaped_truncated}</span>'

# CSS to fix button styling - force white text on dark background for both light/dark themes
button_style_css = HTML('''
<style>
.header-sort-button button,
.header-sort-button .widget-button,
.header-sort-button .jupyter-button,
.header-sort-button button .bp3-button-text,
.header-sort-button button span,
.header-sort-button button p {
    border-radius: 0 !important;
    font-weight: bold !important;
    font-size: 11px !important;
    text-transform: uppercase !important;
    border: none !important;
    border-bottom: 2px solid #555 !important;
    background-color: #333 !important;
    color: #fff !important;
}
.header-sort-button button:hover {
    background-color: #444 !important;
}
.no-gap {
    gap: 0 !important;
}
.no-gap > * {
    margin: 0 !important;
}
</style>
''')
display(button_style_css)

# Get XNAT host for building links
XNAT_HOST = os.environ.get('XNAT_HOST', '')

if not study_df.empty:
    # Recalculate counts
    errors = (study_df['status'] == 'ERROR').sum()
    rejected = (study_df['status'] == 'REJECTED').sum()
    
    # Find AVAILABLE studies missing from XNAT
    available_df = study_df[study_df['status'] == 'AVAILABLE'].copy()
    
    if 'experimentLabel' in available_df.columns:
        def is_missing_from_project(row):
            exp_label = row.get('experimentLabel')
            if pd.isna(exp_label) or exp_label == '' or exp_label is None:
                return True
            return str(exp_label).strip() not in xnat_exp_labels
        
        missing_from_project = available_df[available_df.apply(is_missing_from_project, axis=1)]
    else:
        missing_from_project = pd.DataFrame()
    
    missing_from_project_count = len(missing_from_project)
    
    # Build error details list
    failed_count = errors + rejected + missing_from_project_count
    if failed_count > 0:
        failed_df = study_df[study_df['status'].isin(['ERROR', 'REJECTED'])].copy()
        
        error_details = []
        
        for _, row in failed_df.iterrows():
            status = row.get('status', '')
            if status == 'ERROR':
                status_display = 'Error'
                reason = str(row.get('statusMessage', '')).strip() or 'Unknown'
            else:
                status_display = 'Rejected'
                reason = str(row.get('statusMessage', '')).strip() or 'Rejected by site'
            
            study_uid = str(row.get('studyInstanceUid', '')).strip()
            if not study_uid or study_uid == 'nan' or study_uid == 'None':
                study_uid = 'Unknown'
            
            exp_label = row.get('experimentLabel', '')
            if pd.isna(exp_label):
                exp_label = ''
            
            error_details.append({
                'Experiment Label': str(exp_label),
                'Accession Number': str(row.get('accessionNumber', '')),
                'Study UID': study_uid,
                'Status': status_display,
                'Reason': reason,
            })
        
        for _, row in missing_from_project.iterrows():
            study_uid = str(row.get('studyInstanceUid', '')).strip()
            if not study_uid or study_uid == 'nan' or study_uid == 'None':
                study_uid = 'Unknown'
            
            exp_label = row.get('experimentLabel', '')
            if pd.isna(exp_label):
                exp_label = ''
            
            error_details.append({
                'Experiment Label': str(exp_label),
                'Accession Number': str(row.get('accessionNumber', '')),
                'Study UID': study_uid,
                'Status': 'Missing',
                'Reason': 'Available study missing from Project',
            })
        
        error_table_df = pd.DataFrame(error_details)
        
        # ==================== ERROR DISTRIBUTION TABLE ====================
        if len(error_table_df) > 0:
            error_distribution = error_table_df.groupby('Reason').size().reset_index(name='Count')
            error_distribution = error_distribution.sort_values('Count', ascending=False).reset_index(drop=True)
            
            dist_rows = []
            for _, r in error_distribution.iterrows():
                reason_text = html_module.escape(str(r['Reason']))
                dist_rows.append(f'<tr><td style="padding: 8px 12px; border-bottom: 1px solid #eee; color: #333;">{reason_text}</td><td style="padding: 8px 12px; border-bottom: 1px solid #eee; text-align: center; font-weight: bold; width: 80px; color: #333;">{r["Count"]}</td></tr>')
            
            dist_table_html = f'''
                <div style="background: #f5f5f5; border-radius: 10px; width: 1400px; margin: 10px 5px;">
                    <div style="
                        background: linear-gradient(135deg, #9b59b6 0%, #8e44ad 100%);
                        color: white;
                        padding: 20px;
                        border-radius: 10px 10px 0 0;
                    ">
                        <h3 style="margin: 0; font-size: 18px;">Error Distribution</h3>
                        <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{len(error_distribution)} unique error reasons</p>
                    </div>
                    <div style="background: #f5f5f5; padding: 0; border-radius: 0 0 10px 10px; max-height: 300px; overflow-y: auto;">
                        <table style="width: 100%; border-collapse: collapse; font-size: 12px; background: white;">
                            <thead>
                                <tr style="background: #f8f9fa;">
                                    <th style="padding: 10px 12px; text-align: left; font-size: 11px; text-transform: uppercase; color: #666; border-bottom: 2px solid #ddd;">Reason</th>
                                    <th style="padding: 10px 12px; text-align: center; font-size: 11px; text-transform: uppercase; color: #666; border-bottom: 2px solid #ddd; width: 80px;">Count</th>
                                </tr>
                            </thead>
                            <tbody>
                                {"".join(dist_rows)}
                            </tbody>
                        </table>
                    </div>
                </div>
            '''
            
            display(HTML(dist_table_html))
        
        # ==================== STUDY ERRORS TABLE WITH FILTER ====================
        PAGE_SIZE = 15
        state = {'page': 0, 'sort_col': 'Status', 'sort_asc': True, 'filter': ''}
        
        # Column definitions with Experiment Label
        columns = [
            ('Experiment Label', '12%'),
            ('Accession Number', '12%'),
            ('Study UID', '28%'),
            ('Status', '8%'),
            ('Reason', '40%'),
        ]
        
        filter_input = Text(
            placeholder='Search',
            layout=Layout(width='300px')
        )
        
        prev_btn = Button(description='Previous', button_style='info', layout=Layout(width='100px'))
        next_btn = Button(description='Next', button_style='info', layout=Layout(width='100px'))
        page_label = HTML()
        table_output = Output()
        
        # Create header buttons list first (will be populated below)
        header_buttons = []
        
        def update_header_buttons():
            """Update header button labels with sort arrows."""
            for i, (col_name, width) in enumerate(columns):
                arrow = ''
                if state['sort_col'] == col_name:
                    arrow = ' ↑' if state['sort_asc'] else ' ↓'
                header_buttons[i].description = f'{col_name}{arrow}'
        
        def make_sort_handler(col_name):
            def handler(b):
                if state['sort_col'] == col_name:
                    state['sort_asc'] = not state['sort_asc']
                else:
                    state['sort_col'] = col_name
                    state['sort_asc'] = True
                state['page'] = 0
                update_header_buttons()
                render_table()
            return handler
        
        def render_table():
            table_output.clear_output(wait=True)
            
            # Apply filter across all columns
            filtered_df = error_table_df.copy()
            if state['filter']:
                filter_pattern = state['filter'].replace('*', '.*')
                mask = (
                    filtered_df['Experiment Label'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Accession Number'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Study UID'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Status'].str.contains(filter_pattern, case=False, regex=True, na=False) |
                    filtered_df['Reason'].str.contains(filter_pattern, case=False, regex=True, na=False)
                )
                filtered_df = filtered_df[mask]
            
            sorted_df = filtered_df.sort_values(
                by=state['sort_col'], 
                ascending=state['sort_asc'],
                key=lambda x: x.str.lower() if x.dtype == 'object' else x
            ).reset_index(drop=True)
            
            total_pages = max(1, (len(sorted_df) + PAGE_SIZE - 1) // PAGE_SIZE)
            state['page'] = max(0, min(state['page'], total_pages - 1))
            start = state['page'] * PAGE_SIZE
            end = start + PAGE_SIZE
            page_df = sorted_df.iloc[start:end].copy()
            
            filter_text = f" (filtered: {len(filtered_df)})" if state['filter'] else ""
            page_label.value = f'<span style="font-size: 13px; color: #666;">Page {state["page"] + 1} of {total_pages} ({len(sorted_df)} issues{filter_text})</span>'
            
            prev_btn.disabled = state['page'] == 0
            next_btn.disabled = state['page'] >= total_pages - 1
            
            if len(page_df) > 0:
                # Build data rows only (header is buttons)
                rows = []
                for _, r in page_df.iterrows():
                    status = str(r['Status'])
                    if status == 'Error':
                        bg_color = '#fdeaea'
                        text_color = '#721c24'
                    elif status == 'Rejected':
                        bg_color = '#fff8e6'
                        text_color = '#856404'
                    else:  # Missing
                        bg_color = '#e8daef'
                        text_color = '#4a235a'
                    
                    exp_label = str(r['Experiment Label'])
                    acc = html_module.escape(str(r['Accession Number']))
                    uid = html_module.escape(str(r['Study UID']))
                    stat = html_module.escape(str(r['Status']))
                    reason = str(r['Reason'])
                    reason_truncated = truncate_with_tooltip(reason, 60)
                    
                    # Create experiment label with link if it exists and study is in XNAT
                    if exp_label and exp_label.strip() and exp_label.strip() in xnat_exp_labels:
                        xnat_url = f"{XNAT_HOST}/data/archive/projects/{PROJECT_ID}/experiments/{exp_label.strip()}"
                        exp_display = f'<a href="{xnat_url}" target="_blank" style="color: {text_color}; text-decoration: underline;">{html_module.escape(exp_label)}</a>'
                    else:
                        exp_display = html_module.escape(exp_label) if exp_label else ''
                    
                    rows.append(f'''<tr style="background-color: {bg_color}; color: {text_color};">
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 12%;">{exp_display}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 12%;">{acc}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 28%; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">{uid}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 8%;">{stat}</td>
                        <td style="padding: 8px 12px; border-bottom: 1px solid #eee; width: 40%;">{reason_truncated}</td>
                    </tr>''')
                
                table_html = f'''<table style="width: 100%; border-collapse: collapse; font-size: 12px; table-layout: fixed; background: white; margin: 0; padding: 0;">
                    <tbody>{"".join(rows)}</tbody>
                </table>'''
                
                # Unescape tooltips
                table_html = table_html.replace('&lt;span title=', '<span title=')
                table_html = table_html.replace('style=&quot;cursor: help;&quot;&gt;', 'style="cursor: help;">')
                table_html = table_html.replace('&lt;/span&gt;', '</span>')
                table_html = table_html.replace('&quot;', '"')
                
                with table_output:
                    display(HTML(table_html))
            else:
                with table_output:
                    display(HTML('<div style="padding: 20px; text-align: center; color: #666; background: white;">No matching studies found.</div>'))
        
        # Create header buttons for sorting - use dark background
        for col_name, width in columns:
            arrow = ''
            if state['sort_col'] == col_name:
                arrow = ' ↑' if state['sort_asc'] else ' ↓'
            
            btn = Button(description=f'{col_name}{arrow}')
            btn.layout = Layout(width=width, height='36px', margin='0', padding='0')
            btn.style.button_color = '#333'
            btn.add_class('header-sort-button')
            btn.on_click(make_sort_handler(col_name))
            header_buttons.append(btn)
        
        def on_filter_change(change):
            state['filter'] = change['new']
            state['page'] = 0
            render_table()
        
        def on_prev(b):
            state['page'] -= 1
            render_table()
        
        def on_next(b):
            state['page'] += 1
            render_table()
        
        filter_input.observe(on_filter_change, names='value')
        prev_btn.on_click(on_prev)
        next_btn.on_click(on_next)
        
        render_table()
        
        title_widget = HTML(f'''
            <div style="
                background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%);
                color: white;
                padding: 20px;
                border-radius: 10px 10px 0 0;
            ">
                <h3 style="margin: 0; font-size: 18px;">Missing Studies</h3>
                <p style="margin: 5px 0 0 0; opacity: 0.9; font-size: 13px;">{errors} errors, {rejected} rejected, {missing_from_project_count} missing from project</p>
            </div>
        ''')
        
        # Left side: search box
        left_controls = HBox([filter_input], layout=Layout(align_items='center'))
        
        # Right side: pagination
        right_controls = HBox([prev_btn, page_label, next_btn], layout=Layout(align_items='center', gap='10px'))
        
        # Controls row with space-between to push pagination to right
        controls = HBox(
            [left_controls, right_controls],
            layout=Layout(padding='15px 20px', justify_content='space-between', align_items='center', background='#f5f5f5', width='100%')
        )
        
        # Header row using HBox with buttons - add class for CSS targeting
        header_row = HBox(header_buttons)
        header_row.layout = Layout(width='100%', margin='0', padding='0', background='#333', overflow='hidden')
        header_row.add_class('no-gap')
        
        legend = HTML('''
            <div style="
                padding: 10px 20px;
                font-size: 11px;
                color: #666;
                background: #f8f9fa;
                border-top: 1px solid #eee;
                border-radius: 0 0 10px 10px;
            ">
                <span style="background: #fdeaea; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #721c24;">Error</span> Failed to process
                <span style="margin-left: 20px; background: #fff8e6; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #856404;">Rejected</span> Rejected by site
                <span style="margin-left: 20px; background: #e8daef; padding: 2px 8px; border-radius: 3px; margin-right: 15px; color: #4a235a;">Missing</span> Available study missing from project
            </div>
        ''')
        
        # Table output wrapper with explicit white background
        table_wrapper = VBox([table_output])
        table_wrapper.layout = Layout(background='white', padding='0', margin='0', width='100%', overflow='hidden')
        
        # Table container - header and table together with no gaps
        table_container = VBox([header_row, table_wrapper])
        table_container.layout = Layout(background='#333', padding='0', margin='0', width='100%', overflow='hidden')
        table_container.add_class('no-gap')
        
        full_widget = VBox([
            title_widget,
            controls,
            table_container,
            legend
        ])
        full_widget.layout = Layout(width='1400px', border_radius='10px', overflow='hidden', background='#f5f5f5', margin='10px 5px')
        full_widget.add_class('no-gap')
        
        display(full_widget)
    else:
        display(HTML('''
            <div style="
                background: #d4edda;
                border: 1px solid #28a745;
                border-radius: 6px;
                padding: 15px 20px;
                margin: 10px 5px;
                font-size: 13px;
                color: #155724;
                width: 1400px;
                box-sizing: border-box;
            ">
                <strong>All studies processed successfully.</strong> No errors, rejections, or missing studies.
            </div>
        '''))

In [ ]:
# Missing Series Analysis (displayed at bottom after series data is loaded)
if missing_series_output is not None:
    display(missing_series_output)

In [ ]:
session.disconnect()